<a href="https://colab.research.google.com/github/Rito43/Rito43/blob/main/M23CSA021_DLOps_ClassAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Class Assignment 1: CNN on CIFAR-10 data.

Total Marks: 30


Import the necessary python packages and libraries in the following cell.

In [ ]:
## Import the libraries here in this cell
!pip install torchvision
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.models as models
from torchvision import datasets
from torchvision.models import resnet18






Q1 (a) Print the torch and torchvision version [Marks: 1+1]

In [ ]:
# Your code here

print("PyTorch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

PyTorch version: 2.1.0+cu121
torchvision version: 0.16.0+cu121


Q1 (b) Set the device to cuda, if available. [Marks: 1]

In [ ]:
# device = // Your code here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)



Using device: cuda


Q1 Transform or preprocess the data utilizing the following operations.

*   Normalize data
*   Resize each image to 224 x 224
*   Add Color Jitter with hue and saturation as 0.5
*   Introduce Random horizontal flips
*   Rotate images about 20 degrees


In [ ]:
# transform = // Your code here
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(hue=0.5,saturation=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

Q2 (a) Train and test Resnet18 on Even numbered Classes in CIFAR10 Dataset. [Marks: 5]

In [ ]:
# Filter only even-numbered classes
import torchvision.datasets

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Your code here
even_classes = [i for i in range(0,10,2)]

trainset_even = Subset(trainset,[i for i in range(len(trainset)) if trainset.targets[i] in even_classes])
testset_even = Subset(testset, [i for i in range(len(testset)) if testset.targets[i] in even_classes])
#HINT: Use torch utils Subset class to create a subset of the dataset

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
model = resnet18(weights='DEFAULT').to(device)
model.fc = nn.Linear(model.fc.in_features, len(even_classes)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Q2 (b) Complete the code for backpropagation inside the train function. [Marks: 5]


In [ ]:
def train(model, trainloader, criterion, optimizer, device):

    model.train()

    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in trainloader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Complete the code
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()

        train_loss+=loss.item()


        _, pred = outputs.max(1)
        total_train += labels.size(0)
        correct_train += pred.eq(labels).sum().item()

    train_acc = 100 * correct_train / total_train

    return train_loss/len(trainloader), train_acc

Q2 (c) Complete the test function to test out the model. [Marks: 5]

In [ ]:
def test(model, testloader, criterion, device):
    model.eval()
    correct_test = 0
    total_test = 0

    # Your code here
    with torch.no_grad():
      for inputs,labels in testloader:
        inputs,labels = inputs.to(device),labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs,labels)

        _,pred = outputs.max(1)
        total_test +=labels.size(0)
        correct_test+=pred.eq(labels).sum().item()
    test_accuracy = 100 * correct_test / total_test

    return test_accuracy

Q3 (a) Complete the code to plot the train and test accuracies [Marks: 2]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_accuracies(train_accuracies, test_accuracies, epochs):

    # Your code here
    plt.plot(epochs, train_acc, label='Train Accuracy')
    plt.plot(epochs, test_accuracy, label='Test Accuracy')

    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracies vs Epochs')
    plt.legend()
    plt.show()


Q3 (b) Complete the code to plot a Confusion Matrix (without sklearn) [Marks: 3]

In [ ]:
import seaborn as sns

In [ ]:
def plot_confusion_matrix(model, dataloader, class_names, device):
    model.eval()
    all_labels = []
    all_predictions = []

    num_classes = len(class_names)
    confusion_matrix = torch.zeros(num_classes, num_classes)

    # Your code here
    with torch.no_grad():
      for inputs,labels in dataloader:
        inputs,labels = inputs.to(device),labels.to(device)
        outputs = model(inputs)
        _,predictions = outputs.max(1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())

    confusion_matrix = confusion_matrix(all_labels,all_predictions)


    plt.figure(figsize=(num_classes, num_classes))
    sns.heatmap(confusion_matrix, annot=True, fmt=".0f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
